In [31]:
from datetime import date, timedelta

def get_valid_dates(year):
    start_date = date(year, 1, 1)
    end_date = date(year, 12, 31)
    delta = timedelta(days=1)
    valid_dates = []

    current_date = start_date
    while current_date <= end_date:
        valid_dates.append(current_date.strftime('%Y-%m-%d'))
        current_date += delta

    return valid_dates

year = 2022  # Change this to the desired year
valid_dates = get_valid_dates(year)
print(valid_dates)


['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08', '2022-01-09', '2022-01-10', '2022-01-11', '2022-01-12', '2022-01-13', '2022-01-14', '2022-01-15', '2022-01-16', '2022-01-17', '2022-01-18', '2022-01-19', '2022-01-20', '2022-01-21', '2022-01-22', '2022-01-23', '2022-01-24', '2022-01-25', '2022-01-26', '2022-01-27', '2022-01-28', '2022-01-29', '2022-01-30', '2022-01-31', '2022-02-01', '2022-02-02', '2022-02-03', '2022-02-04', '2022-02-05', '2022-02-06', '2022-02-07', '2022-02-08', '2022-02-09', '2022-02-10', '2022-02-11', '2022-02-12', '2022-02-13', '2022-02-14', '2022-02-15', '2022-02-16', '2022-02-17', '2022-02-18', '2022-02-19', '2022-02-20', '2022-02-21', '2022-02-22', '2022-02-23', '2022-02-24', '2022-02-25', '2022-02-26', '2022-02-27', '2022-02-28', '2022-03-01', '2022-03-02', '2022-03-03', '2022-03-04', '2022-03-05', '2022-03-06', '2022-03-07', '2022-03-08', '2022-03-09', '2022-03-10', '2022-03-11', '2022-03-12', '2022

In [ ]:
# !pip install MechanicalSoup
# !pip install PyPDF2
# !pip install tabula-py pandas openpyxl

import requests
import pandas as pd
import mechanicalsoup
from bs4 import BeautifulSoup
import tabula
from urllib.parse import urljoin
import PyPDF2

browser = mechanicalsoup.Browser()

url = 'https://report.grid-india.in/psp_report.php'
page = browser.get(url, verify=False)





In [30]:
data_list = []
dont_check = ['2020-07-12']
# valid_dates = ['2014-04-11']
for input_date in valid_dates:
    try:

        #   if date in dont_check:
        #     data_list.append({"Date": date, "Total Power": 'NULL'})

          form = page.soup.find("form", class_="date-form")
          date_input = form.find("input", type="date")
          date_input["value"] = input_date

          response = browser.submit(form, page.url, verify=False)




          soup = BeautifulSoup(response.text, "html.parser")
          pdf_links = soup.find_all("a", href=True)
          pdf_link = None

          for link in pdf_links:
              if link["href"].endswith(".pdf"):
                  pdf_link = link["href"]
                  print("Found PDF link:", pdf_link)
                  break
                

          if not pdf_link:
              print("No PDF link found.")
              continue




          # Step 6: Download the PDF file
          base_link = "https://report.grid-india.in/"
          down_link = urljoin(base_link, pdf_link)
          pdf_response = requests.get(down_link, verify=False)
          pdf_filename = "temp.pdf"
          with open(pdf_filename, "wb") as pdf_file:
              pdf_file.write(pdf_response.content)




          #Use PyPDF2 to extract content from page 2
          pdf_filename = "temp.pdf"  # Replace with the actual filename
          output_pdf_filename = "page2.pdf"  # Replace with the desired output filename

          with open(pdf_filename, "rb") as pdf_file:
              pdf_reader = PyPDF2.PdfReader(pdf_file)
              pdf_writer = PyPDF2.PdfWriter()

              # Add only page 2 to the new PDF writer
              pdf_writer.add_page(pdf_reader.pages[1])

              # Save the new PDF containing only page 2
              with open(output_pdf_filename, "wb") as output_pdf_file:
                  pdf_writer.write(output_pdf_file)

          print("Page 2 extracted and saved to", output_pdf_filename)





          pdf_path = "page2.pdf"

          tables = tabula.read_pdf(pdf_path, pages="all", multiple_tables=True)

          # Assume you want the first table (use indexing if there are multiple tables)
          table_data = tables[0]



          # Convert the table data into a pandas DataFrame
          df = pd.DataFrame(table_data)

          # Save the DataFrame into an Excel file
          excel_output = "output.xlsx"
          df.to_excel(excel_output, index=False, engine="openpyxl")
          print("Data extracted and saved to", excel_output)

          # Load the output Excel file
          output_excel_file = "output.xlsx"  # Replace with the actual filename
          df_output = pd.read_excel(output_excel_file)

          # Extract the date and total power columns
          date = input_date  # Assuming the date is in the first row, first column
          total_power = df_output.iloc[0, 6]  # Assuming total power is in the second row, seventh column

          data_list.append({"Date": input_date, "Total Power": total_power})

    except Exception as e:
        data_list.append({"Date": input_date, "Total Power": 'NULL'})
        print("exception")
    
    

new_df = pd.DataFrame(data_list)

C:\Users\tyagi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\tyagi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\tyagi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/adv

No PDF link found.
No PDF link found.
No PDF link found.
No PDF link found.
No PDF link found.
No PDF link found.
No PDF link found.
No PDF link found.
No PDF link found.


C:\Users\tyagi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\tyagi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\tyagi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/adv

No PDF link found.
No PDF link found.
No PDF link found.
No PDF link found.
No PDF link found.
No PDF link found.


C:\Users\tyagi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\tyagi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\tyagi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/adv

...

Got stderr: Aug 25, 2023 2:36:33 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 25, 2023 2:36:33 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 25, 2023 2:36:33 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 25, 2023 2:36:33 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 25, 2023 2:36:33 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode

C:\Users\tyagi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\tyagi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. 

Data extracted and saved to output.xlsx
Found PDF link: ./ReportData/Daily Report/PSP Report/2013-2014/April 2013/06.04.13_NLDC_PSP.pdf
Page 2 extracted and saved to page2.pdf


Got stderr: Aug 25, 2023 2:36:35 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 25, 2023 2:36:35 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 25, 2023 2:36:35 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 25, 2023 2:36:35 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 25, 2023 2:36:35 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode

C:\Users\tyagi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\tyagi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. 

Data extracted and saved to output.xlsx
Found PDF link: ./ReportData/Daily Report/PSP Report/2013-2014/April 2013/07.04.13_NLDC_PSP.pdf
Page 2 extracted and saved to page2.pdf


KeyboardInterrupt: 

In [25]:
new_df.head()

,Date,Total Power
0,2014-01-01,NULL
1,2014-01-02,NULL
2,2014-01-03,NULL
3,2014-01-04,NULL
4,2014-01-05,121664


In [26]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363 entries, 0 to 362
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         363 non-null    object
 1   Total Power  357 non-null    object
dtypes: object(2)
memory usage: 5.8+ KB


In [27]:
new_df.to_excel('2022.xlsx', index=False, header=True)

In [61]:
len(valid_dates)

365